In [1]:
import pandas as pd

In [29]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

def random_neg(df, gene, meta,positive_triples,):
    df = df.copy()
    negative_samples = []
    # 生成3个破坏头实体的负样本
    # for _, row in df.iterrows():
    #     from_entity = row['from']
    #     rel = row['rel']
    #     to_entity = row['to']
        
    #     # 生成3个破坏头实体的负样本
    #     for _ in range(6):
    #         neg_from = random.choice(list(gene))
    #         if (neg_from, rel, to_entity) not in positive_triples:
    #             negative_samples.append({'from': neg_from, 'rel': rel, 'to': to_entity, 'label': 0})
    
    for _, row in df.iterrows():
        from_entity = row['from']
        rel = row['rel']
        to_entity = row['to']
        
        # 生成3个破坏头实体的负样本
        for _ in range(6):
            neg_to = random.choice(list(meta))
            if (from_entity, rel, neg_to) not in positive_triples:
                negative_samples.append({'from': from_entity, 'rel': rel, 'to': neg_to, 'label': 0})

    return negative_samples

def negative_sampling_df(positive_triples, df, gene, meta):    
    # 创建正样本的三元组集合，便于后续检查是否为正样本 
    neg = random_neg(df, gene, meta, positive_triples)
    # 将正样本加入到负样本中，标记为label=1
    for _, row in df.iterrows():
        neg.append({'from': row['from'], 'rel': row['rel'], 'to': row['to'], 'label': 1})
    
    # 返回包含正负样本的 DataFrame
    return pd.DataFrame(neg)

def negative_sampling(train, gene, meta):    
    # 创建正样本的三元组集合，便于后续检查是否为正样本
    all = train
    positive_triples = set(zip(all['from'], all['rel'], all['to']))
    train_plus_neg = negative_sampling_df(positive_triples, train, gene, meta)
    
    # 返回包含正负样本的 DataFrame
    return train_plus_neg

In [30]:
kg = pd.read_csv('/home/shengkun/repo/DeepME/data/KG/ECO/kg.txt',sep='\t',names=['from','rel','to'])
ent = set(kg['from']).union(set(kg['to']))
all_gene = [x for x in ent if x.startswith('ECO_Protein')]
gene_w_rule = set(kg[kg['rel'] == 'Catalyzes']['from'])
gene_wo_rule = set(all_gene) - gene_w_rule
meta = set(kg[kg['rel'].isin(['Consumes','Produces'])]['to'])
meta = [x for x in meta if x.endswith('_c')]
len(all_gene), len(gene_w_rule), len(meta), len(gene_wo_rule)

(4136, 1451, 1044, 2685)

In [34]:
eco_before_2023 = pd.read_csv('/home/shengkun/repo/DeepME/data/me_data/train_data/eco_before_2023_all.txt',sep='\t',names=['from','rel','to'])
from collections import Counter

head_from_counter = Counter(eco_before_2023['from'])
tail_from_counter = Counter(eco_before_2023['to'])

test = pd.read_csv('./test.txt',sep='\t',names=['from','rel','to','label'])
test['head_count'] = [head_from_counter[x] for x in list(test['from'])]
test['tail_count'] = [tail_from_counter[x] for x in list(test['to'])]
test

,from,rel,to,label
0,ECO_Protein:P0ABP8,knock out,Metabolite:ac_c,0
1,ECO_Protein:P0AB80,knock out,Metabolite:ac_c,0
2,ECO_Protein:P37689,knock out,Metabolite:ac_c,0
3,ECO_Protein:P17443,knock out,Metabolite:arg__L_c,0
4,ECO_Protein:P09831,knock out,Metabolite:arg__L_c,0
...,...,...,...,...
2182,ECO_Protein:P11667,overexpress,Metabolite:arg__L_c,1
2183,ECO_Protein:P0A6C5,knock out,Metabolite:arg__L_c,1
2184,ECO_Protein:P21169,knock out,Metabolite:arg__L_c,1
2185,ECO_Protein:P24169,knock out,Metabolite:arg__L_c,1


In [36]:
eco_before_2023

,from,rel,to
0,ECO_Protein:P0AFG3,knock out,Metabolite:4abut_c
1,ECO_Protein:P52643,knock out,Metabolite:fum_c
2,ECO_Protein:P16095,knock out,Metabolite:ser__L_c
3,ECO_Protein:P0A9M8,knock out,Metabolite:trp__L_c
4,ECO_Protein:P15993,knock out,Metabolite:tyr__L_c
...,...,...,...
1287,ECO_Protein:P0A6B4,overexpress,Metabolite:3hpp_c
1288,ECO_Protein:P0A9H3,overexpress,Metabolite:lys__L_c
1289,ECO_Protein:P0A9D4,overexpress,Metabolite:cys__L_c
1290,ECO_Protein:P00957,overexpress,Metabolite:5aop_c


In [27]:
unseen_met_triple = test[(test['tail_count'] == 0) & (test['label'] == 1)][['from','rel','to']]
low_met_triple = test[(test['tail_count']>0) & (test['tail_count']<=5) & (test['label'] == 1)][['from','rel','to']]
high_met_triple = test[(test['tail_count']>5) & (test['label'] == 1)][['from','rel','to']]

unseen_gene_triple = test[(test['head_count'] == 0) & (test['label'] == 1)][['from','rel','to']]
low_gene_triple = test[(test['head_count']>0) & (test['head_count']<=5) & (test['label'] == 1)][['from','rel','to']]
high_gene_triple = test[(test['head_count']>5) & (test['label'] == 1)][['from','rel','to']]

In [20]:
len(unseen_met_triple), len(low_met_triple), len(high_met_triple)

(51, 74, 189)

In [32]:
len(unseen_gene_triple), len(low_gene_triple), len(high_gene_triple)

(76, 127, 111)

In [21]:
test_unseen_met_head = negative_sampling(unseen_met_triple, gene_w_rule, meta)
test_unseen_met_head.to_csv('./representional_split/test_unseen_met_head.txt',sep='\t',index=False,header=False)
test_low_met_head = negative_sampling(low_met_triple, gene_w_rule, meta)
test_low_met_head.to_csv('./representional_split/test_low_met_head.txt',sep='\t',index=False,header=False)
test_high_met_head = negative_sampling(high_met_triple, gene_w_rule, meta)
test_high_met_head.to_csv('./representional_split/test_high_met_head.txt',sep='\t',index=False,header=False)

In [33]:
test_unseen_gene_tail = negative_sampling(unseen_gene_triple, gene_w_rule, meta)
test_unseen_gene_tail.to_csv('./representional_split/test_unseen_gene_tail.txt',sep='\t',index=False,header=False)
test_low_gene_tail = negative_sampling(low_gene_triple, gene_w_rule, meta)
test_low_gene_tail.to_csv('./representional_split/test_low_gene_tail.txt',sep='\t',index=False,header=False)
test_high_gene_tail = negative_sampling(high_gene_triple, gene_w_rule, meta)
test_high_gene_tail.to_csv('./representional_split/test_high_gene_tail.txt',sep='\t',index=False,header=False)